In [1]:
import os
import cv2
import numpy as np
import imutils
from tqdm import tqdm
from pathlib import Path

In [2]:
# Constants
IMG_SIZE = 300
CLAHE = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

In [3]:
def crop_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    if len(cnts) == 0:
        return img  # Return original if no contour found
    c = max(cnts, key=cv2.contourArea)
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    ADD_PIXELS = 0
    cropped = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    return cropped

In [4]:
def apply_clahe(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    clahe_applied = CLAHE.apply(gray)
    return cv2.cvtColor(clahe_applied, cv2.COLOR_GRAY2RGB)  # Convert back to RGB

In [5]:
def process_images(input_root, output_root):
    input_root = Path(input_root)
    output_root = Path(output_root)
    output_root.mkdir(parents=True, exist_ok=True)

    for category in tqdm(os.listdir(input_root), desc="Processing categories"):
        input_category_path = input_root / category
        output_category_path = output_root / category
        output_category_path.mkdir(exist_ok=True)

        for img_name in os.listdir(input_category_path):
            img_path = input_category_path / img_name
            img = cv2.imread(str(img_path))
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            cropped = crop_img(img)
            enhanced = apply_clahe(cropped)
            resized = cv2.resize(enhanced, (IMG_SIZE, IMG_SIZE))
            final_img = cv2.cvtColor(resized, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(output_category_path / img_name), final_img)

In [6]:
# Run preprocessing on the dataset
process_images("C:/EfficientNetB3/Training", "C:/EfficientNetB3/cropped/Training")
process_images("C:/EfficientNetB3/Testing", "C:/EfficientNetB3/cropped/Testing")

Processing categories: 100%|█████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.18s/it]
